In [ ]:
# Random Forest para Classificação do Jogo da Velha
## Comparação com Decision Tree Classifier

Este notebook implementa um modelo Random Forest para classificar estados do jogo da velha e compara com o Decision Tree Classifier.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')

# Configuração para melhor visualização
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)

print("Bibliotecas importadas com sucesso!")


In [ ]:
# Carregar o dataset
velha = pd.read_csv('IA_trab-lucas_branch/amostras_.csv', sep=';')

print(f"Dataset shape: {velha.shape}")
print(f"\nPrimeiras 5 linhas:")
print(velha.head())

print(f"\nClasses disponíveis: {velha['classe'].unique()}")
print(f"\nDistribuição das classes:")
print(velha['classe'].value_counts())

# Separar features e target
X = velha.drop(columns=['classe'])
y = velha['classe']

print(f"\nFeatures: {list(X.columns)}")
print(f"Target shape: {y.shape}")


In [ ]:
# Divisão estratificada dos dados
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.1, 
    random_state=42, 
    stratify=y
)

print(f"Treino: {X_train.shape[0]} amostras")
print(f"Teste: {X_test.shape[0]} amostras")
print(f"\nDistribuição no conjunto de teste:")
print(y_test.value_counts())

# Visualização da distribuição
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
y_train.value_counts().plot(kind='bar', color='skyblue')
plt.title('Distribuição - Conjunto de Treino')
plt.xlabel('Classe')
plt.ylabel('Frequência')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
y_test.value_counts().plot(kind='bar', color='lightcoral')
plt.title('Distribuição - Conjunto de Teste')
plt.xlabel('Classe')
plt.ylabel('Frequência')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
# Modelo Random Forest com parâmetros padrão
rf_model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1  # Usar todos os cores disponíveis
)

print("Treinando o modelo Random Forest...")
rf_model.fit(X_train, y_train)
print("Treinamento concluído!")

# Predições
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"\nAcurácia: {accuracy:.4f} ({accuracy*100:.2f}%)")


In [ ]:
# Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusão:")
print(cm)

# Visualização da matriz de confusão
plt.figure(figsize=(8, 6))
disp = ConfusionMatrixDisplay(
    confusion_matrix=cm, 
    display_labels=['Em_jogo', 'Possibilidade_de_fim_de_jogo', 'Fim_de_jogo']
)
disp.plot(cmap='Blues', values_format='d')
plt.title('Matriz de Confusão - Random Forest')
plt.tight_layout()
plt.show()


In [ ]:
# Relatório de classificação
print("Relatório de Classificação:")
print(classification_report(y_test, y_pred, 
                          target_names=['Em_jogo', 'Possibilidade_de_fim_de_jogo', 'Fim_de_jogo']))

# Análise detalhada por classe
print("\nAnálise Detalhada por Classe:")
classes = ['Em_jogo', 'Possibilidade_de_fim_de_jogo', 'Fim_de_jogo']
for i, classe in enumerate(classes):
    tp = cm[i, i]
    fp = cm[:, i].sum() - tp
    fn = cm[i, :].sum() - tp
    tn = cm.sum() - tp - fp - fn
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    print(f"\n{classe}:")
    print(f"  Precisão: {precision:.4f} ({precision*100:.2f}%)")
    print(f"  Recall: {recall:.4f} ({recall*100:.2f}%)")
    print(f"  F1-Score: {f1:.4f} ({f1*100:.2f}%)")
    print(f"  Verdadeiros Positivos: {tp}")
    print(f"  Falsos Positivos: {fp}")
    print(f"  Falsos Negativos: {fn}")


In [ ]:
# Validação cruzada
print("Validação Cruzada (5-fold):")
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='accuracy')
print(f"Scores de validação cruzada: {cv_scores}")
print(f"Média: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# Visualização dos scores de validação cruzada
plt.figure(figsize=(8, 5))
plt.plot(range(1, 6), cv_scores, 'bo-', linewidth=2, markersize=8)
plt.axhline(y=cv_scores.mean(), color='r', linestyle='--', label=f'Média: {cv_scores.mean():.4f}')
plt.fill_between(range(1, 6), 
                 cv_scores.mean() - cv_scores.std(), 
                 cv_scores.mean() + cv_scores.std(), 
                 alpha=0.2, color='red')
plt.xlabel('Fold')
plt.ylabel('Acurácia')
plt.title('Scores de Validação Cruzada')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
# Definir o grid de parâmetros para otimização
import numpy as np
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

print("Grid de parâmetros definido:")
for param, values in param_grid.items():
    print(f"  {param}: {values}")

print(f"\nTotal de combinações: {np.prod([len(v) for v in param_grid.values()])}")


In [ ]:
# Executar GridSearchCV
print("Iniciando GridSearchCV...")
print("Isso pode levar alguns minutos...")

# Garantir que as importações estejam disponíveis
try:
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import GridSearchCV
    print("Bibliotecas importadas com sucesso!")
except ImportError as e:
    print(f"Erro na importação: {e}")

# Criar o modelo base
rf_base = RandomForestClassifier(random_state=42, n_jobs=-1)

# Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid,
    cv=5,  # 5-fold cross validation
    scoring='accuracy',
    n_jobs=-1,  # Usar todos os cores disponíveis
    verbose=1  # Mostrar progresso
)

# Executar a busca
grid_search.fit(X_train, y_train)

print("\nGridSearchCV concluído!")
print(f"Melhor score: {grid_search.best_score_:.4f}")
print(f"Melhores parâmetros: {grid_search.best_params_}")


In [ ]:
# Análise dos resultados do GridSearchCV
print("=== ANÁLISE DOS RESULTADOS ===")

# Top 10 melhores combinações
results_df = pd.DataFrame(grid_search.cv_results_)
top_results = results_df.nlargest(10, 'mean_test_score')[
    ['params', 'mean_test_score', 'std_test_score']
]

print("\nTop 10 melhores combinações:")
for i, (_, row) in enumerate(top_results.iterrows(), 1):
    print(f"\n{i}. Score: {row['mean_test_score']:.4f} (+/- {row['std_test_score']*2:.4f})")
    print(f"   Parâmetros: {row['params']}")

# Comparação com modelo original
print(f"\n=== COMPARAÇÃO ===")
print(f"Modelo original (parâmetros padrão):")
print(f"  Acurácia: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")
print(f"  Parâmetros: n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features='sqrt'")

print(f"\nMelhor modelo encontrado:")
print(f"  Acurácia: {grid_search.best_score_:.4f} (+/- {grid_search.cv_results_['std_test_score'][grid_search.best_index_]*2:.4f})")
print(f"  Parâmetros: {grid_search.best_params_}")

improvement = ((grid_search.best_score_ - cv_scores.mean()) / cv_scores.mean()) * 100
print(f"\nMelhoria: {improvement:.2f}%")


In [ ]:
# Treinar modelo final com melhores parâmetros
print("Treinando modelo final com melhores parâmetros...")

# Usar o melhor modelo encontrado
best_rf_model = grid_search.best_estimator_

# Fazer predições no conjunto de teste
y_pred_best = best_rf_model.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)

print(f"Acurácia no conjunto de teste: {accuracy_best:.4f} ({accuracy_best*100:.2f}%)")

# Comparação final
print(f"\n=== COMPARAÇÃO FINAL ===")
print(f"Modelo original: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Modelo otimizado: {accuracy_best:.4f} ({accuracy_best*100:.2f}%)")

test_improvement = ((accuracy_best - accuracy) / accuracy) * 100
print(f"Melhoria no teste: {test_improvement:.2f}%")


In [ ]:
# Visualização da comparação de performance
plt.figure(figsize=(12, 5))

# Gráfico 1: Comparação de acurácias
plt.subplot(1, 2, 1)
models = ['Original', 'Otimizado']
accuracies = [accuracy, accuracy_best]
colors = ['lightcoral', 'lightgreen']

bars = plt.bar(models, accuracies, color=colors, alpha=0.8)
plt.title('Comparação de Acurácia')
plt.ylabel('Acurácia')
plt.ylim(0, 1)

# Adicionar valores nas barras
for bar, acc in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')

plt.grid(True, alpha=0.3, axis='y')

# Gráfico 2: Comparação de validação cruzada
plt.subplot(1, 2, 2)
cv_original = cv_scores.mean()
cv_optimized = grid_search.best_score_

cv_models = ['Original', 'Otimizado']
cv_scores_comp = [cv_original, cv_optimized]
cv_colors = ['lightcoral', 'lightgreen']

bars2 = plt.bar(cv_models, cv_scores_comp, color=cv_colors, alpha=0.8)
plt.title('Validação Cruzada (5-fold)')
plt.ylabel('Acurácia Média')
plt.ylim(0, 1)

# Adicionar valores nas barras
for bar, acc in zip(bars2, cv_scores_comp):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')

plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


In [ ]:
# Matriz de confusão do modelo otimizado
cm_best = confusion_matrix(y_test, y_pred_best)

# Comparação das matrizes de confusão
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Matriz original
disp1 = ConfusionMatrixDisplay(
    confusion_matrix=cm, 
    display_labels=['Em_jogo', 'Possibilidade_de_fim_de_jogo', 'Fim_de_jogo']
)
disp1.plot(ax=axes[0], cmap='Blues', values_format='d')
axes[0].set_title('Matriz de Confusão - Modelo Original')

# Matriz otimizada
disp2 = ConfusionMatrixDisplay(
    confusion_matrix=cm_best, 
    display_labels=['Em_jogo', 'Possibilidade_de_fim_de_jogo', 'Fim_de_jogo']
)
disp2.plot(ax=axes[1], cmap='Greens', values_format='d')
axes[1].set_title('Matriz de Confusão - Modelo Otimizado')

plt.tight_layout()
plt.show()

# Relatório de classificação do modelo otimizado
print("Relatório de Classificação - Modelo Otimizado:")
print(classification_report(y_test, y_pred_best, 
                          target_names=['Em_jogo', 'Possibilidade_de_fim_de_jogo', 'Fim_de_jogo']))
